# 程序员的数学基础

## 二进制

二进制左移一位就是乘以2

In [8]:
a = 1
b = a << 1
assert(b == a*2)
c = a << 3
assert(c == a*(2**3))

二进制右移一位就是除以2求整数商

In [20]:
a = 53001
b = a >> 1
assert(b == (a // 2))
c = a >> 4
assert(c == (a // (2**4)))

1. 位与：两个位必须全部为1结果才为1
2. 位或：两个位只要有一个为1，结果就为1
3. 异或：两个位相同，结果为0；两个位不同，结果为1

## 余数

同余定理：<br>
两个整数a,b,如果他们除以同一个数m后余数相等，我们就可以说a和b对于模m同余。<br>
同余定理其实就是用来分类的。

In [22]:
a = 5
print("5 除以 2 余 ", 5%2)

5 除以 2 余  1


取余操作也可以当作hash来用

## 迭代法
迭代法就是不断用旧的变量计算出新的变量的值<br>
它很适合用循环语句来实现<br>
二分法就是迭代法的一个经典例子<br>
下面的例子就是使用二分法来计算平方根，这里面不断迭代的变量就是mid<br>

In [57]:
import math

def it_sqrt(num, precision = 10**-10, max_it_count=10**6):
    if num < 0:
        raise ValueError("num should > 0")
    small = 0
    big = num
    for i in range (max_it_count):
        mid = (big + small) / 2
        loss = mid**2 - num
        #print("i=%d mid=%f loss = %f" %(i, mid, loss))
        if abs(loss) < precision:
            print("get res i=", i)
            return (mid, loss)
        elif loss > 0: # mid is too big, pick small ~ mid
            big = mid
        else: # mid is too small, pick mid ~ big
            small = mid
    return(mid, loss)
    raise RuntimeError("can not get result in %d loss=%f" % (max_it_count, loss))
num = 5000000000000
print(math.sqrt(num))
res, loss = it_sqrt(num)
print("result=%f loss=%f"% (res,loss))

2236067.9774997896
result=2236067.977500 loss=-0.000977
